### Requirements
- Split the dataset into two tables
- Damage Stats: 1 containing: Name, Category, Phy, Mag, Fire, Ligh, Holy
- Level Requirements: 1 containing: Name, Str, Dex, Int, Fai, Arc
- For Damage Stats, if you look at the data in Phy, Mag, Fire, Ligh, Holy:
- -  the first value shows the attack damage 
- - the second value shows the damage resistance
- - A dash “-” means 0, i.e. no damage 
- For Level Requirements, for Str, Dex, Int, Fai, Arc:
- the first value shows the level required
- the second value shows the weapon scaling rating
- A dash “-” means 0, i.e. no requirement or scaling 
- Pivot the two datasets:
 - For Damage Stats,1 column containing Phy, Mag, Fire, Ligh, Holy and 1 for the values
 - For Level Requirements,1 column containing Str, Dex, Int, Fai, Arc and 1 for the values
 - Split the pivot values into 2 columns for both tables:
 - For Damage Stats label the first column “Attack Damage”, and the second “Damage Resistance”
 - For Level Requirements label the first column “Required Level”, and the second “Attribute Scaling” 
 - Replace the “-” values with zero in the columns: “Attack Damage”, “Damage Resistance”, “Required Level” and change the datatype to whole numbers (integer)
 - Find the total “Attack Damage” and total “Required Level” for all weapons and join the datasets together.
 - Rank the weapons by total attack damage, grouped by the total required attribute. 
 - Filter for the number 1 rank and output the data


In [1]:
import os
import pandas as pd

In [2]:
# Download sheet
# url = ' https://drive.google.com/file/d/1hWHg2TOW9eUZVlcmGopg1Biz7wjN9UF0/view'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# pd.read_csv(path).to_csv('preppin data week 37.csv')

df = pd.read_csv('preppin data week 37.csv').drop(columns='Unnamed: 0')

In [3]:
df.columns = [i.strip().lower().replace(' ','_') for i in df.columns]

df.head()

,name,phy,mag,fire,ligh,holy,critboost,str,dex,int,fai,arc,wgt,skill,category,category_url,last_updated
0,Dagger,74 35,- 20,- 20,- 20,- 20,130 15,5 D,9 C,- -,- -,- -,1.5,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
1,Parrying Dagger,75 35,- 20,- 20,- 20,- 20,110 15,5 E,14 C,- -,- -,- -,1.5,- Kick,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
2,Misericorde,92 36,- 21,- 21,- 21,- 21,140 15,7 E,12 D,- -,- -,- -,2.0,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
3,Great Knife,75 35,- 20,- 20,- 20,- 20,110 15,6 E,12 C,- -,- -,- -,1.5,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
4,Bloodstained Dagger,81 36,- 21,- 21,- 21,- 21,110 15,9 C,12 E,- -,- -,- -,2.0,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022


 - Split the dataset into two tables
 - Damage Stats: 1 containing: Name, Category, Phy, Mag, Fire, Ligh, Holy
 - Level Requirements: 1 containing: Name, Str, Dex, Int, Fai, Arc

In [4]:
# Create 2 dfs
damage_df = df.loc[:,['name', 'phy', 'mag', 'fire', 'ligh', 'holy']]
level_requirement_df = df.loc[:,['name','str', 'dex', 'int', 'fai', 'arc']]

In [5]:
# expand columns

def expand_clm(df,clm): #damage DF
    new_df = df[clm].str.split(expand=True)
    new_df.columns = [f'{clm}_damage',f'{clm}_expand']
    df = df.merge(new_df,left_index=True, right_index=True)

    return df

def expand_clm2(df,clm): #level Requirement
    new_df = df[clm].str.split(expand=True)
    new_df.columns = [f'{clm}_level_required',f'{clm}_scaling_rating']
    df = df.merge(new_df,left_index=True, right_index=True)

    return df

for i in damage_df.columns: #damage DF
    if i != 'name':
        damage_df = expand_clm(damage_df,i)
        damage_df.drop(columns=i,inplace=True)

for i in level_requirement_df.columns:  #level Requirement
    if i != 'name':
        level_requirement_df = expand_clm2(level_requirement_df,i)
        level_requirement_df.drop(columns=i,inplace=True)

# Replace with zeros
level_requirement_df.replace('-',0,inplace=True)
damage_df.replace('-',0,inplace=True)

In [6]:
# Pivot tables

damage_df = pd.melt(damage_df, id_vars='name', value_vars=['phy_damage', 'phy_expand', 'mag_damage', 'mag_expand',
       'fire_damage', 'fire_expand', 'ligh_damage', 'ligh_expand',
       'holy_damage', 'holy_expand'])

level_requirement_df=level_requirement_df.melt(id_vars='name',value_vars=['str_level_required', 'str_scaling_rating',
       'dex_level_required', 'dex_scaling_rating', 'int_level_required',
       'int_scaling_rating', 'fai_level_required', 'fai_scaling_rating',
       'arc_level_required', 'arc_scaling_rating'])

#Expand Columns: damage_df
new_df = damage_df.variable.str.split('_',expand=True)
new_df.columns = ['attack_damage','damage_resistance']
damage_df = damage_df.merge(new_df,left_index=True,right_index=True)
damage_df.drop(columns=['variable'],inplace=True)

#Expand Columns: level_requirements
new_df = level_requirement_df.variable.str.split('_',expand=True,n=1)
new_df.columns = ['required_level','attribute_scaling']
level_requirement_df=level_requirement_df.merge(new_df,left_index=True,right_index=True)
level_requirement_df.drop(columns=['variable'],inplace=True)

In [7]:
damage_df.value=damage_df.value.astype('int64')

In [8]:
# damage_df.loc[damage_df.damage_resistance=='damage'].set_index(['name','attack_damage']).sort_index()

damage_df.set_index(['name','attack_damage','damage_resistance']).sort_index()

value
name                   attack_damage damage_resistance       
Alabaster Lord's Sword fire          damage                 0
                                     expand                27
                       holy          damage                 0
                                     expand                27
                       ligh          damage                 0
...                                                       ...
Zweihander             ligh          expand                40
                       mag           damage                 0
                                     expand                40
                       phy           damage               141
                                     expand                67

[2560 rows x 1 columns]

In [9]:
level_requirement_df.loc[level_requirement_df.attribute_scaling=='level_required'].set_index(['name','required_level']).sort_index()

value attribute_scaling
name                   required_level                        
Alabaster Lord's Sword arc                0    level_required
                       dex               12    level_required
                       fai                0    level_required
                       int               18    level_required
                       str               16    level_required
...                                     ...               ...
Zweihander             arc                0    level_required
                       dex               11    level_required
                       fai                0    level_required
                       int                0    level_required
                       str               19    level_required

[1280 rows x 2 columns]